In [ ]:
import matplotlib.pyplot as plt
import time
from Params import *
from WIMPFuncs import *
from numpy import *
from LabFuncs import *
from Like import *
ne = 200
E_r = logspace(-3.0,3.0,ne)
t = JulianDay(9,12,2018,18.0)*ones(shape=ne)

xmin = 1.0e-3
xmax = 1000.0
ymax = 1.0e8
ymin = 1.0e-6

Nuc = He4
HaloModel = SHM
Loc = Boulby
DM = WIMP(10.0,1.0e-47)
Exposure = 1.0

m_vals = logspace(log10(0.5),3.0,50)
sigma_vals = logspace(-49.0,-40.0,500)
Volume = 1000.0
TotTime = 3.0


Exposure = Volume*TotTime*(0.16/1000.0)
Loc = Params.Boulby

nam = "Pixel"

t0 = time.time()

DLHe,DLF = CYGNUSLimit("test/CYGNUS1000.txt",m_vals,sigma_vals,Volume,TotTime,   ReadOut_Name=nam)

t1 = time.time()

print 'Time elapsed:',t1-t0



CYGNUS limits
m_chi =  0.5 sigma_p =  0.0 0.0 6.28561385271
m_chi =  0.583899311869 sigma_p =  0.0 0.0 6.28561385271
m_chi =  0.681876812802 sigma_p =  0.0 0.0 6.28561385271
m_chi =  0.796294803549 sigma_p =  0.0 0.0 6.28561385271
m_chi =  0.929911975673 sigma_p =  0.0 0.0 6.28561385271
m_chi =  1.08594992539 sigma_p =  0.0 0.0 6.28561385271
m_chi =  1.26817082832 sigma_p =  0.0 0.0 6.28561385271
m_chi =  1.48096814797 sigma_p =  0.0 0.0 6.28561385271
m_chi =  1.729472565 sigma_p =  0.0 0.0 6.28561385271
m_chi =  2.0196756812 sigma_p =  0.0 0.0 6.28561385271
m_chi =  2.3585744809 sigma_p =  1.36228676414e-41 4.56185746041 6.28561385271


In [ ]:
# PLOT
plt.rcParams['axes.linewidth'] = 2
fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111)

plt.loglog(m_vals,DLHe,'r-')
plt.loglog(m_vals,DLF,'r-')

data = loadtxt("../data/CYGNUS1000-"+nam+".txt")
m,F,He = data[0,:], data[1,:], data[2,:]
plt.loglog(data[0,:],data[1,:],'b-')
plt.loglog(data[0,:],data[2,:],'b-')

plt.show()

HaloModel = SHM
Expt = Params.Detector(1.8,200.0,Params.He4,Loc,0.16*3,ne=20,nt=1,nside=0,ReadOut_Name = nam)

sigma_p = 1.0e-45
m_chi = 10.0
Signal = Params.WIMP(m_chi,1.0e-45)
RD_wimp = LabFuncs.BinEvents(Expt,WIMPFuncs.WIMPRate,Signal,HaloModel)
Signal.RecoilDistribution(RD_wimp/1.0e-45)
        
Background = NeutrinoFuncs.GetNuFluxes(Expt.EnergyThreshold,Expt.Nucleus)
n_bg = Background.NumberOfNeutrinos
R_bg = Background.Normalisations
R_bg_err = Background.Uncertainties
RD_nu = zeros(shape=(Expt.TotalNumberOfBins,n_bg))
for i in range(0,n_bg):
    RD_nu[:,i] = LabFuncs.BinEvents(Expt,NeutrinoFuncs.NuRate,Background,i)
    RD_nu[:,i] *= (1.0/R_bg[i])
Background.RecoilDistribution(RD_nu)
X_in1 = zeros(shape=(n_bg+1))
N_bg = zeros(shape=Expt.TotalNumberOfBins)
for i in range(0,n_bg):
    N_bg = N_bg + R_bg[i]*Background.RD[:,i]
    
    

N_signal = Signal.RD*sigma_p
N_obs = N_signal + N_bg

X_in1 = append(log10(sigma_p),R_bg)
L1 = llhood1(X_in1,N_obs,Signal,Background)
print(L1)

X_in0 = R_bg
L0 = llhood0(X_in0,N_obs,Signal,Background)
print(L0)
print 2*(L0-L1)

In [19]:
lnGF(Background.Normalisations,Background.Normalisations,Background.Uncertainties*Background.Normalisations)

-27.727487978003321

In [22]:
Background.Uncertainties*Background.Normalisations

array([  2.39400000e+03,   1.03200000e+05,   4.28500000e+01,
         2.63500000e+00])